# Convolutional Neural Network

### Importing the libraries

In [288]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

In [289]:
tf.__version__

'2.13.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [301]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('data\\train',
                                                 target_size = (100, 100),
                                                 batch_size = 150,
                                                 class_mode = "categorical")

Found 31405 images belonging to 27 classes.


### Preprocessing the Test set

In [302]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('data/valid',
                                            target_size = (100, 100),
                                            batch_size = 150,
                                            class_mode = 'categorical')

Found 7401 images belonging to 27 classes.


## Part 2 - Building the CNN


### Initialising the CNN

In [303]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [304]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', input_shape=[100,100,3]))

### Step 2 - Pooling

In [305]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))

### Adding a second convolutional layer

In [306]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))


### Step 3 - Flattening

In [307]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [308]:
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))



cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(Dropout(0.5))

### Step 5 - Output Layer

In [309]:
cnn.add(tf.keras.layers.Dense(units=27, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [313]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True)

### Training the CNN on the Training set and evaluating it on the Test set

In [314]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 40, callbacks=[checkpoint])

Epoch 1/40
210/210 [==============================] - 527s 3s/step - loss: 1.3696 - accuracy: 0.5271 - val_loss: 0.8126 - val_accuracy: 0.7458
Epoch 2/40
210/210 [==============================] - 520s 2s/step - loss: 1.0226 - accuracy: 0.6444 - val_loss: 0.5966 - val_accuracy: 0.8026
Epoch 3/40
210/210 [==============================] - 519s 2s/step - loss: 0.8246 - accuracy: 0.7169 - val_loss: 0.4477 - val_accuracy: 0.8602
Epoch 4/40
210/210 [==============================] - 521s 2s/step - loss: 0.6797 - accuracy: 0.7678 - val_loss: 0.4081 - val_accuracy: 0.8895
Epoch 5/40
210/210 [==============================] - 519s 2s/step - loss: 0.5842 - accuracy: 0.8033 - val_loss: 0.2843 - val_accuracy: 0.9118
Epoch 6/40
210/210 [==============================] - 520s 2s/step - loss: 0.5021 - accuracy: 0.8311 - val_loss: 0.2720 - val_accuracy: 0.9242
Epoch 7/40
210/210 [==============================] - 7594s 36s/step - loss: 0.4336 - accuracy: 0.8543 - val_loss: 0.2541 - val_accuracy: 0.92

KeyboardInterrupt: 

###### Part 4 - Making a single prediction

In [315]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('best_model.h5')

In [319]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)  # You can add more preprocessing options if needed

# Assume 'test_dir' is the directory containing your test images
test_generator = test_datagen.flow_from_directory(
    'data/valid',
    target_size=(100, 100),
    batch_size=15,
    class_mode='categorical',  # Assuming you have categorical labels
    shuffle=False  # Important: Do not shuffle the test data
)

# Evaluate the model on the test data
test_loss, test_accuracy = cnn.evaluate_generator(test_generator, steps=len(test_generator))

# Print the evaluation results
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Found 7401 images belonging to 27 classes.


C:\Users\bnnar\AppData\Local\Temp\ipykernel_22392\973395172.py:16: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = cnn.evaluate_generator(test_generator, steps=len(test_generator))


Test Loss: 0.10554616153240204
Test Accuracy: 0.9713551998138428
